# Bank Customer Churn Prediction

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv('Churn_Modelling.csv')

In [3]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
df.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [5]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [6]:
def find_value_counts(df):
    for i in df.columns:
        print(f'{df[i].value_counts()}')
        print("\n")
find_value_counts(df)

RowNumber
1        1
6671     1
6664     1
6665     1
6666     1
        ..
3334     1
3335     1
3336     1
3337     1
10000    1
Name: count, Length: 10000, dtype: int64


CustomerId
15634602    1
15667932    1
15766185    1
15667632    1
15599024    1
           ..
15599078    1
15702300    1
15660735    1
15671390    1
15628319    1
Name: count, Length: 10000, dtype: int64


Surname
Smith       32
Scott       29
Martin      29
Walker      28
Brown       26
            ..
Izmailov     1
Bold         1
Bonham       1
Poninski     1
Burbidge     1
Name: count, Length: 2932, dtype: int64


CreditScore
850    233
678     63
655     54
705     53
667     53
      ... 
404      1
351      1
365      1
417      1
419      1
Name: count, Length: 460, dtype: int64


Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64


Gender
Male      5457
Female    4543
Name: count, dtype: int64


Age
37    478
38    477
35    474
36    456
34    447
     ... 
92      2
82  

In [7]:
df.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [8]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [9]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [10]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [12]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [13]:
df['Gender'].replace({'Female':1,'Male':0},inplace=True)
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,1,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,1,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,1,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,1,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,1,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,0,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,0,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,1,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,0,42,3,75075.31,2,1,0,92888.52,1


In [14]:
df=pd.get_dummies(df,columns=['Geography'],dtype='int')

In [15]:
df

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,1,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,2,15647311,Hill,608,1,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,Onio,502,1,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,4,15701354,Boni,699,1,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,5,15737888,Mitchell,850,1,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,0,39,5,0.00,2,1,0,96270.64,0,1,0,0
9996,9997,15569892,Johnstone,516,0,35,10,57369.61,1,1,1,101699.77,0,1,0,0
9997,9998,15584532,Liu,709,1,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,9999,15682355,Sabbatini,772,0,42,3,75075.31,2,1,0,92888.52,1,0,1,0


In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
columns=['CreditScore','Age','Tenure','Balance','EstimatedSalary']
df[columns]=scaler.fit_transform(df[columns])
df

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,0.538,1,0.324324,0.2,0.000000,1,1,1,0.506735,1,1,0,0
1,2,15647311,Hill,0.516,1,0.310811,0.1,0.334031,1,0,1,0.562709,0,0,0,1
2,3,15619304,Onio,0.304,1,0.324324,0.8,0.636357,3,1,0,0.569654,1,1,0,0
3,4,15701354,Boni,0.698,1,0.283784,0.1,0.000000,2,0,0,0.469120,0,1,0,0
4,5,15737888,Mitchell,1.000,1,0.337838,0.2,0.500246,1,1,1,0.395400,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,0.842,0,0.283784,0.5,0.000000,2,1,0,0.481341,0,1,0,0
9996,9997,15569892,Johnstone,0.332,0,0.229730,1.0,0.228657,1,1,1,0.508490,0,1,0,0
9997,9998,15584532,Liu,0.718,1,0.243243,0.7,0.000000,1,0,1,0.210390,1,1,0,0
9998,9999,15682355,Sabbatini,0.844,0,0.324324,0.3,0.299226,2,1,0,0.464429,1,0,1,0


In [17]:
df['NumOfProducts'].value_counts()

NumOfProducts
1    5084
2    4590
3     266
4      60
Name: count, dtype: int64

In [18]:
df['NumOfProducts'].replace({1:0,2:1,3:1,4:1},inplace=True)

In [19]:
df['NumOfProducts'].value_counts()

NumOfProducts
0    5084
1    4916
Name: count, dtype: int64

In [20]:
df.Exited.value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [21]:
# doing undersampling
df_churn_yes=df[df['Exited']==1]
df_churn_no=df[df['Exited']==0]
df_churn_no=df_churn_no.sample(n=2037)
df_new=pd.concat([df_churn_yes,df_churn_no],axis=0)
df_new

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,0.538,1,0.324324,0.2,0.000000,0,1,1,0.506735,1,1,0,0
2,3,15619304,Onio,0.304,1,0.324324,0.8,0.636357,1,1,0,0.569654,1,1,0,0
5,6,15574012,Chu,0.590,0,0.351351,0.8,0.453394,1,1,0,0.748797,1,0,0,1
7,8,15656148,Obinna,0.052,1,0.148649,0.4,0.458540,1,1,0,0.596733,1,0,1,0
16,17,15737452,Romeo,0.606,0,0.540541,0.1,0.528513,0,1,0,0.025433,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7924,7925,15613337,Gallo,0.966,0,0.391892,0.2,0.000000,1,1,1,0.911268,0,1,0,0
4131,4132,15738634,Yuan,0.366,0,0.391892,0.9,0.332196,0,1,1,0.688489,0,1,0,0
3928,3929,15609545,Azubuike,0.396,0,0.148649,0.5,0.332577,0,0,1,0.885114,0,1,0,0
2887,2888,15604314,Webb,0.706,1,0.108108,0.1,0.387931,0,1,0,0.318560,0,0,1,0


In [22]:
df_new['Exited'].value_counts()

Exited
1    2037
0    2037
Name: count, dtype: int64

In [23]:
df_new

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,0.538,1,0.324324,0.2,0.000000,0,1,1,0.506735,1,1,0,0
2,3,15619304,Onio,0.304,1,0.324324,0.8,0.636357,1,1,0,0.569654,1,1,0,0
5,6,15574012,Chu,0.590,0,0.351351,0.8,0.453394,1,1,0,0.748797,1,0,0,1
7,8,15656148,Obinna,0.052,1,0.148649,0.4,0.458540,1,1,0,0.596733,1,0,1,0
16,17,15737452,Romeo,0.606,0,0.540541,0.1,0.528513,0,1,0,0.025433,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7924,7925,15613337,Gallo,0.966,0,0.391892,0.2,0.000000,1,1,1,0.911268,0,1,0,0
4131,4132,15738634,Yuan,0.366,0,0.391892,0.9,0.332196,0,1,1,0.688489,0,1,0,0
3928,3929,15609545,Azubuike,0.396,0,0.148649,0.5,0.332577,0,0,1,0.885114,0,1,0,0
2887,2888,15604314,Webb,0.706,1,0.108108,0.1,0.387931,0,1,0,0.318560,0,0,1,0


In [24]:
X=df_new.drop(columns=['RowNumber','CustomerId','Surname','Exited'])
Y=df_new['Exited']


In [25]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,0.538,1,0.324324,0.2,0.000000,0,1,1,0.506735,1,0,0
2,0.304,1,0.324324,0.8,0.636357,1,1,0,0.569654,1,0,0
5,0.590,0,0.351351,0.8,0.453394,1,1,0,0.748797,0,0,1
7,0.052,1,0.148649,0.4,0.458540,1,1,0,0.596733,0,1,0
16,0.606,0,0.540541,0.1,0.528513,0,1,0,0.025433,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
7924,0.966,0,0.391892,0.2,0.000000,1,1,1,0.911268,1,0,0
4131,0.366,0,0.391892,0.9,0.332196,0,1,1,0.688489,1,0,0
3928,0.396,0,0.148649,0.5,0.332577,0,0,1,0.885114,1,0,0
2887,0.706,1,0.108108,0.1,0.387931,0,1,0,0.318560,0,1,0


In [26]:
Y

0       1
2       1
5       1
7       1
16      1
       ..
7924    0
4131    0
3928    0
2887    0
1374    0
Name: Exited, Length: 4074, dtype: int64

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [28]:
print(X_train.shape,X_test.shape)

(3259, 12) (815, 12)


In [29]:
model=keras.Sequential([
    keras.layers.Dense(10,input_shape=(12,),activation='relu'),
    keras.layers.Dense(1,activation='sigmoid'),
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.fit(X_train,Y_train,epochs=100)

Epoch 1/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5816 - loss: 0.6746
Epoch 2/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6460 - loss: 0.6389
Epoch 3/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6494 - loss: 0.6270
Epoch 4/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6631 - loss: 0.6136
Epoch 5/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6731 - loss: 0.6059
Epoch 6/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 976us/step - accuracy: 0.6835 - loss: 0.5909
Epoch 7/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7024 - loss: 0.5845
Epoch 8/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6803 - loss: 0.5999
Epoch 9/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6747 - loss: 0.5960
Epoch 10/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - accuracy: 0.6889 - loss: 0.5892
Epoch 11/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step - accuracy: 0.6938 - loss: 0.5814
Epoch 12/100
102/102 ━━━━━━━━━━━━━━

102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7329 - loss: 0.5396
Epoch 69/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7165 - loss: 0.5532
Epoch 70/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7178 - loss: 0.5536
Epoch 71/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7119 - loss: 0.5469
Epoch 72/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7389 - loss: 0.5380
Epoch 73/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7201 - loss: 0.5485
Epoch 74/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7230 - loss: 0.5474
Epoch 75/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7312 - loss: 0.5322
Epoch 76/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7365 - loss: 0.5405
Epoch 77/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7287 - loss: 0.5437
Epoch 78/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7346 - loss: 0.5366
Epoch 79/100
102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [30]:
model.evaluate(X_test,Y_test)

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7210 - loss: 0.5595  


[0.5702299475669861, 0.7116564512252808]

In [31]:
Y_test[0:5]

868     1
3467    0
7701    1
8018    1
6051    0
Name: Exited, dtype: int64

In [32]:
np.argmax(model.predict(X_test[0:1]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


0

In [33]:
yp=model.predict(X_test)
y_pred=[]
for i in yp:
    if (i>0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)
y_pred[:5]

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


[1, 0, 0, 0, 1]

In [34]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.76      0.68      0.72       441
           1       0.67      0.75      0.70       374

    accuracy                           0.71       815
   macro avg       0.71      0.71      0.71       815
weighted avg       0.72      0.71      0.71       815



In [35]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,y_pred)
cm

array([[301, 140],
       [ 95, 279]], dtype=int64)

In [44]:
def ANN(X_train,Y_train,X_test,Y_test,loss,weights):
    
    model=keras.Sequential([
    keras.layers.Dense(12,input_shape=(12,),activation='relu'),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dense(1,activation='sigmoid'),
    ])

    model.compile(
        optimizer='adam',
        loss=loss,
        metrics=['accuracy']
    )
    
    if weights==-1:
        model.fit(X_train,Y_train,epochs=100)
    else:
        model.fit(X_train,Y_train,epochs=100,class_weight=weights)
    
    y_preds=model.predict(X_test)
    y_preds=np.round(y_preds)
    print("Classification Report : \n",classification_report(Y_test,y_preds))
    
    return y_preds

## Improving the F1 Score using Sampling Techniques

In [45]:
df

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,0.538,1,0.324324,0.2,0.000000,0,1,1,0.506735,1,1,0,0
1,2,15647311,Hill,0.516,1,0.310811,0.1,0.334031,0,0,1,0.562709,0,0,0,1
2,3,15619304,Onio,0.304,1,0.324324,0.8,0.636357,1,1,0,0.569654,1,1,0,0
3,4,15701354,Boni,0.698,1,0.283784,0.1,0.000000,1,0,0,0.469120,0,1,0,0
4,5,15737888,Mitchell,1.000,1,0.337838,0.2,0.500246,0,1,1,0.395400,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,0.842,0,0.283784,0.5,0.000000,1,1,0,0.481341,0,1,0,0
9996,9997,15569892,Johnstone,0.332,0,0.229730,1.0,0.228657,0,1,1,0.508490,0,1,0,0
9997,9998,15584532,Liu,0.718,1,0.243243,0.7,0.000000,0,0,1,0.210390,1,1,0,0
9998,9999,15682355,Sabbatini,0.844,0,0.324324,0.3,0.299226,1,1,0,0.464429,1,0,1,0


In [46]:
df['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

###  1st using Under sampling

In [47]:
df_exited_0=df[df['Exited']==0]
df_exited_1=df[df['Exited']==1]

In [48]:
df_exited_0_under=df_exited_0.sample(n=2037)

In [49]:
df_new_under=pd.concat([df_exited_0_under,df_exited_1],axis=0)
df_new_under['Exited'].value_counts()

Exited
0    2037
1    2037
Name: count, dtype: int64

In [50]:
X=df_new_under.drop(columns=['RowNumber','CustomerId','Surname','Exited'])
Y=df_new_under['Exited']

In [51]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [58]:
y_pred=ANN(X_train,Y_train,X_test,Y_test,'binary_crossentropy',-1)

Epoch 1/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6105 - loss: 0.6609
Epoch 2/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6906 - loss: 0.5978
Epoch 3/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7071 - loss: 0.5670
Epoch 4/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7166 - loss: 0.5587
Epoch 5/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7151 - loss: 0.5602
Epoch 6/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7307 - loss: 0.5425
Epoch 7/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7209 - loss: 0.5484
Epoch 8/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7235 - loss: 0.5453
Epoch 9/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 990us/step - accuracy: 0.7230 - loss: 0.5417
Epoch 10/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7245 - loss: 0.5391
Epoch 11/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7215 - loss: 0.5383
Epoch 12/100
399/399 ━━━━━━━━━━━━━━━━━━

399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7562 - loss: 0.4941
Epoch 69/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7561 - loss: 0.4937
Epoch 70/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7498 - loss: 0.5007
Epoch 71/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7638 - loss: 0.4855
Epoch 72/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7525 - loss: 0.4912
Epoch 73/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7585 - loss: 0.4922
Epoch 74/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7542 - loss: 0.4958
Epoch 75/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7572 - loss: 0.4926
Epoch 76/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7538 - loss: 0.4971
Epoch 77/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7603 - loss: 0.4869
Epoch 78/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7508 - loss: 0.4976
Epoch 79/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 

### 2nd using Over Sampling

In [59]:
df_exited_1_over=df_exited_1.sample(7963,replace=True)
df_new_over=pd.concat([df_exited_0,df_exited_1_over],axis=0)
df_new_over['Exited'].value_counts()

Exited
0    7963
1    7963
Name: count, dtype: int64

In [60]:
X=df_new_over.drop(columns=['RowNumber','CustomerId','Surname','Exited'])
Y=df_new_over['Exited']

In [61]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [62]:
y_pred=ANN(X_train,Y_train,X_test,Y_test,'binary_crossentropy',-1)

Epoch 1/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5868 - loss: 0.6665
Epoch 2/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 996us/step - accuracy: 0.6839 - loss: 0.5896
Epoch 3/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7129 - loss: 0.5507
Epoch 4/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7210 - loss: 0.5469
Epoch 5/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7357 - loss: 0.5340
Epoch 6/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7321 - loss: 0.5340
Epoch 7/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7391 - loss: 0.5285
Epoch 8/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 970us/step - accuracy: 0.7347 - loss: 0.5250
Epoch 9/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7366 - loss: 0.5278
Epoch 10/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7389 - loss: 0.5239
Epoch 11/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7415 - loss: 0.5173
Epoch 12/100
399/399 ━━━━━━━━━━━━━━━━

399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7657 - loss: 0.4824
Epoch 69/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7662 - loss: 0.4889
Epoch 70/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 954us/step - accuracy: 0.7603 - loss: 0.4900
Epoch 71/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 954us/step - accuracy: 0.7702 - loss: 0.4783
Epoch 72/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 992us/step - accuracy: 0.7616 - loss: 0.4927
Epoch 73/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7680 - loss: 0.4793
Epoch 74/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7581 - loss: 0.4863
Epoch 75/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7648 - loss: 0.4880
Epoch 76/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7676 - loss: 0.4837
Epoch 77/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7738 - loss: 0.4784
Epoch 78/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.7653 - loss: 0.4787
Epoch 79/100
399/399 ━━━━━━━━━━━━━━━━

### 3rd using SMOTE

In [63]:
from imblearn.over_sampling import SMOTE

In [64]:
smote=SMOTE(sampling_strategy='minority')

In [65]:
df

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,0.538,1,0.324324,0.2,0.000000,0,1,1,0.506735,1,1,0,0
1,2,15647311,Hill,0.516,1,0.310811,0.1,0.334031,0,0,1,0.562709,0,0,0,1
2,3,15619304,Onio,0.304,1,0.324324,0.8,0.636357,1,1,0,0.569654,1,1,0,0
3,4,15701354,Boni,0.698,1,0.283784,0.1,0.000000,1,0,0,0.469120,0,1,0,0
4,5,15737888,Mitchell,1.000,1,0.337838,0.2,0.500246,0,1,1,0.395400,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,0.842,0,0.283784,0.5,0.000000,1,1,0,0.481341,0,1,0,0
9996,9997,15569892,Johnstone,0.332,0,0.229730,1.0,0.228657,0,1,1,0.508490,0,1,0,0
9997,9998,15584532,Liu,0.718,1,0.243243,0.7,0.000000,0,0,1,0.210390,1,1,0,0
9998,9999,15682355,Sabbatini,0.844,0,0.324324,0.3,0.299226,1,1,0,0.464429,1,0,1,0


In [66]:
X=df.drop(columns=['RowNumber','CustomerId','Surname','Exited'])
Y=df['Exited']

In [67]:
X_sm,Y_sm=smote.fit_resample(X,Y)

In [68]:
Y_sm.value_counts()

Exited
1    7963
0    7963
Name: count, dtype: int64

In [69]:
X_train,X_test,Y_train,Y_test=train_test_split(X_sm,Y_sm,test_size=0.2,stratify=Y_sm)

In [70]:
y_preds=ANN(X_train,Y_train,X_test,Y_test,'binary_crossentropy',-1)

Epoch 1/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5786 - loss: 0.6752
Epoch 2/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6902 - loss: 0.5963
Epoch 3/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7004 - loss: 0.5731
Epoch 4/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7202 - loss: 0.5557
Epoch 5/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 970us/step - accuracy: 0.7274 - loss: 0.5405
Epoch 6/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7241 - loss: 0.5459
Epoch 7/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7347 - loss: 0.5384
Epoch 8/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7355 - loss: 0.5310
Epoch 9/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 996us/step - accuracy: 0.7382 - loss: 0.5222
Epoch 10/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7415 - loss: 0.5298
Epoch 11/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 974us/step - accuracy: 0.7310 - loss: 0.5352
Epoch 12/100
399/399 ━━━━━━━━━━━━━━

399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7637 - loss: 0.4795
Epoch 69/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7687 - loss: 0.4830
Epoch 70/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7593 - loss: 0.4920
Epoch 71/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7664 - loss: 0.4811
Epoch 72/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7651 - loss: 0.4828
Epoch 73/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7708 - loss: 0.4762
Epoch 74/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7657 - loss: 0.4792
Epoch 75/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7663 - loss: 0.4895
Epoch 76/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7686 - loss: 0.4727
Epoch 77/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7668 - loss: 0.4825
Epoch 78/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7635 - loss: 0.4815
Epoch 79/100
399/399 ━━━━━━━━━━━━━━━━━━━━ 0s 